In [8]:
all = [var for var in globals() if var[0] != '_']
for var in all:
    del globals()[var]
del all, var

In [9]:
import numpy as np
import jax
import jax.numpy as jnp
import torch

### < 클래스화를 '굳이' 왜 하려 하는가? 즉 왜 굳이 객체지향프로그래밍(OOP)을 하려 하는가? >
많은 서로 다른 함수에서 특정 구조가 '반복'되는 경우, 스크립트의 글자수를 줄이기 위해 해당 구조를 하나의 '무언가'로 통일하고 싶을 수 있다.  
(대개 이런 경우는 어떤 작업을 수행하는 데 있어 여러 가지 옵션(방법론)이 있을 때이다, 수치최적화가 딱 이런 경우에 해당한다.)  
(이거 말고도 뭐 FEM으로 치면 mesh를 "어떤 element로 구성할지", "solver는 어떤 solver를 쓸지" 등 다양한 선택지가 있는 이런 경우가 전형적 예시가 되겠다.)  
이럴 경우 클래스를 쓴다. 클래스의 '상속'이 이러한 '반복'을 대신하는 것이다.  
단 클래스화를 하려면, 이런 서로 다른 함수들 사이에 '공통 구조(반복 구조)'는 무엇인지, '독자 구조'는 무엇인지 정확하게 알고 있어야 한다.  
만약 공통 구조를 제대로 캐치하지 못해 부모 클래스에 그것을 충실히 반영하지 못하면, 서로 다른 자식 클래스들 사이에 원래의 함수 스크립트와 똑같이 반복되는 구조가 다시금 재현된다.  
즉, "씨바 이럴거면 시간 써가면서 클래스화 왜 했냐 ㅄ아?" 이 소리가 나오게 되는 거다.  
하지만 많은 함수가 하나의 스크립트에 존재할 때, 그것들의 공통구조/독자구조를 정확히 구별하는 것은 매우 어렵다.  
따라서 초반에는 일단 최대한 여러 부모 클래스를 만들어 이것들을 관리하고, 추후 코드를 더 디벨롭하면서 통일시킬 수 있는 클래스들을 통일하는 방향으로 코드를 수정해나가야 한다.  
비제약 최적화에 대한 클래스화부터 한 후 제약 최적화에 대한 클래스화를 한다.

### PyTorch AD 쓰는 버전(25.11.24에 f(numpy 연산용)와 f_torch(torch AD 계산용)를 분리하여 받는 구조로 수정)
굳이 이렇게 한 이유는, 원래 내가 짠 오리지널 버전의 optimization 코드가 전부 다 numpy 연산 기반으로 구성되어있기 때문이다. 게다가 torch의 AD는 torch 데이터타입에 대해서만 연산 가능하다.  
오리지널 버전의 optimization 코드에 torch의 AD 함수를 insert하려면 torch 데이터타입과 numpy 데이터타입이 호환되지 않기 때문에,  
AD 함수에서 나온 gradient 값들을 전부 다 numpy 데이터타입으로 바꾼 뒤 return해야 한다.  
만약 numpy 데이터타입으로 바꾸지 않고 그냥 return하게 되면 return된 torch 데이터 타입의 gradient가 optimization 내 다른 곳에서 numpy 연산과 만날 때 타입 미스매치로 인한 연산 에러가 발생된다.  
따라서 전체 코드에서 쓰이는 데이터타입을 numpy 데이터타입으로 통일하든 torch 데이터타입으로 통일하든 반드시 통일은 해야 한다.  
근데 거의 95% 이상 numpy 연산으로 짜여졌기 때문에 당연히 numpy 데이터타입으로 통일하는 편이 나은 선택이다.  
따라서 전체 코드는 numpy 연산으로 구성하되 torch AD 연산 시에만 예외적으로, 연산에 쓰이는 함수를 torch 데이터타입으로 연산하는 함수를 쓰도록 하고 그 output을 numpy 데이터타입으로 변환하여 return하게끔 한다.  
따라서 사용자는 UnconOpt_torch 클래스 내 optimizer 사용 시 반드시 목적함수 및 제약함수를 numpy 연산 버전과 torch 연산 버전으로 둘 다 준비하여 입력해주어야 한다.  
이것이 아래 코드의 grad_torch 함수 및 UnconOpt_torch 클래스에 구현되어있다.

In [10]:
# torch의 grad를 이용한 AD 구현 ... np.ndarray 타입의 x를 받아 torch.tensor 타입으로 변경 후 AD를 한 결과를 다시 np.ndarray로 변환 후 리턴
def grad_torch(f_torch, x):
    x = torch.tensor(x, requires_grad=True)
    fx = f_torch(x).backward()
    # fx.backward()
    dfdx = x.grad.detach().numpy().astype(np.float64)
    return dfdx

# ----------------------------------------- Parent Optimizer Class -----------------------------------------
class UnconOpt_torch: # 비제약 Optimizer(SDM, CGM, ...)에 대한 부모 클래스 정의
    '''
    자식 클래스들은 부모 클래스의 모든 함수 및 속성들을 상속받는다.  
    따라서 부모 클래스는 최대한 모든 자식 클래스에서 공통적으로 가지는 함수 및 속성들을 가지고 있어야 한다.  
    (바꿔 말하면, 기존에 클래스 없이 함수로만 짜여진 모듈을 클래스화하고 싶으면 그 함수들의 공통점을 최대한 솎아내서 부모 클래스가 가지도록 클래스를 설계해야 함)  
    자식 클래스는 부모 클래스에는 없는 자기 자신만의 함수 및 속성들을 가지고 있어야 한다(또는 부모 클래스의 함수를 이름만 같은 완전 새로운 자기만의 함수로 재정의할 수 있다).  
    '''
    def __init__(self, f, f_torch, x0, tol=1e-6, max_iter=1000): # 모든 클래스 메서드는 첫번째 arg로 instance 자기 자신을 뜻하는 'self'를 받아야 한다. 다른 단어를 써도 되지만 관례적으로 거의 무조건 self를 쓴다.
        '''
        Initialization method : 해당 클래스 인스턴스 생성(초기화) 시 부여되는 속성을 정의.  
        반드시 self 변수(해당 클래스의 인스턴스 생성 시 인스턴스 자기 자신을 의미)를 첫번째 arg로 받아야 함
        '''
        if not callable(f):
            raise ValueError('Please input f as function type !')
        else:
            self.f = f
            self.f_torch = f_torch

        if (not isinstance(x0, np.ndarray)) or (len(x0.shape) != 1):
            raise ValueError('Please input x0 as 1D ndarray type !')
        else:
            x0 = x0.astype(np.float64)
            # self.x = torch.tensor(x0, requires_grad=False)
            self.x = x0
        
        self.f_val = self.f(self.x)
        if not np.isfinite(self.f_val).all():
            raise ValueError('f(x0) is not finite. Check it again !')
        else:
            self.tol = float(tol)

        self.grad_f_val = grad_torch(self.f_torch, self.x)
        ### Check ||∇f(x0)||
        if np.linalg.norm(self.grad_f_val) < self.tol: # Check optimality
            print(f'Since ||grad(x0)|| = {np.linalg.norm(self.grad_f_val)} < {self.tol}, x0 : {self.x} is optimum point !')
            return [self.x], [self.f_val], [self.grad_f_val]
        else:
            print(f'Since ||grad(x0)|| = {np.linalg.norm(self.grad_f_val)} > {self.tol}, x0 : {self.x} is not an optimum point. Optimization available !')
            self.max_iter = max_iter
            self.k = 0
            pass

        # Optimization Log
        self.log_x = [self.x]
        self.log_f_val = [self.f_val]
        self.log_grad_f = [self.grad_f_val]

    # Search direction method(오버라이드 by 자식 메서드)
    def search_direction(self):
        raise NotImplementedError # 반드시 자식 메서드에서 구현되어야 함을 개발자에게 표시
    
    # interpol_alpha ⊂ bracketing_alpha ⊂ wolfe_strong_interpol
    def interpol_alpha(self, f, f_torch, x_cur, p_cur, a, b):
        # bracketing_alpha에서 계속해서 업데이트되는 구간(alpha_lo, alpha_hi)에 대해, 양단 점을 기반으로 2(3)차 함수로 근사하여 alpha_new 찾는 함수
        # 이 alpha_new는 bracketing_alpha에서 구간을 새로 업데이트할지 말지, 업데이트한다면 어떤 방향으로 업데이트할 지 판단할 때 사용
        phi_a = f(x_cur + a*p_cur)
        phi_b = f(x_cur + b*p_cur)
        dphi_a = grad_torch(f_torch, x_cur + a*p_cur)@p_cur
        
        dem = 2*(phi_b - phi_a - dphi_a*(b - a)) # 2차 함수 근사식 기반 minimum point 계산식의 분모
        
        if (abs(dem) < 1e-12) | (not np.isfinite(dem)): # 분모 수치적으로 불안정하면
            alpha_min = .5*(a + b) # 그냥 구간의 중심점을 minimum point로 상정하고 return
            return alpha_min
        else: # 분모 수치적으로 안정하면
            num = dphi_a*(b - a) # 분자 계산해주고
            alpha_min = a - num/dem # 2차근사식 minimum point 계산식 기반 minimum point 구해준다.
            alpha_min = np.clip(alpha_min, a + 0.1*(b - a), b - 0.1*(b - a)) # 만약 minimum point가 너무 작은 값이면(유의미한 point 이동 못 이뤄냄) 구간의 10% 지점으로, 너무 큰 값(너무 많이 point 이동해도 문제)이면 구간의 90% 지점으로 치환한다.
            return alpha_min   
    
    # bracketing_alpha ⊂ wolfe_strong_interpol
    def bracketing_alpha(self, f, f_torch, x_cur, p_cur, c2_dphi0, phi_armijo, alpha_lo, alpha_hi): 
        # wolfe_strong_interpol에서 확정된 '큰' 골짜기 구간을 기반으로 alpha_optm 찾는 함수
        # 여기서 추가적으로 구간을 더 작게 업데이트할 수도 있음 -> '작은' 골짜기 구간
        phi_lo = f(x_cur+alpha_lo*p_cur)

        for _ in range(50):
            alpha_new = self.interpol_alpha(f, f_torch, x_cur, p_cur, alpha_lo, alpha_hi) # 다항식 보간함수로 골짜기 구간에서의 최소추정점 alpha_new 구함
            phi_new = f(x_cur + alpha_new*p_cur) # alpha_new에서의 함수값
            dphi_new = grad_torch(f_torch, x_cur + alpha_new*p_cur)@p_cur # alpha_new에서의 기울기

            if (phi_new > phi_armijo(alpha_new)) | (phi_new >= phi_lo): # alpha_new에서의 함수값이 오히려 증가했다
                alpha_hi = alpha_new # alpha_optm은 alpha_lo와 alpha_new 사이 존재 -> '작은' 골짜기 구간 [alpha_lo, alpha_new]로 업뎃
                # alpha_lo unchanged
            else:
                if abs(dphi_new) <= c2_dphi0: # alpha_new에서의 함수값이 감소했고 기울기까지 작다
                    alpha_optm = alpha_new # alpha_new가 alpha_optm
                    return alpha_optm
                elif dphi_new > 0: # alpha_new에서의 함수값이 감소했는데 기울기는 여전히 양수다
                    alpha_hi = alpha_new # alpha_optm은 alpha_lo와 alpha_new 사이 존재 -> '작은' 골짜기 구간 [alpha_lo, alpha_new]로 업뎃
                    # alpha_lo unchanged
                else: # alpha_new에서의 함수값이 감소했는데 기울기가 음수다
                    alpha_lo = alpha_new # alpha_optm은 alpha_new와 alpha_hi 사이 존재 -> '작은' 골짜기 구간 [alpha_new, alpha_hi]로 업뎃
                    # alpha_hi unchanged
            
            phi_lo = f(x_cur + alpha_lo*p_cur) # 업뎃된 alpha_lo에서의 함수값 계산
            
            if abs(alpha_hi - alpha_lo) < 1e-8: # 만약 구간이 충분히 줄어들었으면 그냥 탈출해서 구간의 절반지점을 alpha_optm으로 return
                break
        
        alpha_optm = 0.5*(alpha_lo + alpha_hi)
        return alpha_optm

    # wolfe_strong_interpol - 확실한 '큰' 골짜기 구간을 찾아 거기서 alpha_optm을 찾는 함수
    def wolfe_strong_interpol(self, f, f_torch, x_cur, f_cur, grad_f_cur, p_cur, c2=0.9):
        c1 = 1e-4 # Armijo 조건, Curvature 조건용 factors
        alpha_try_old, alpha_try = 0, 1 # Initial bracket of alpha

        phi0 = f_cur # Armijo 함수 생성용
        dphi0 = grad_f_cur@p_cur # Armijo 함수 생성용 및 Curvature 조건 비교용

        phi_armijo = lambda alpha : phi0 + c1*alpha*dphi0 # Armijo 람다 함수 정의

        for _ in range(50):
            x_try = x_cur + alpha_try*p_cur
            phi_try = f(x_try)
            dphi_try = grad_torch(f_torch, x_try)@p_cur

            phi_armijo_try = phi_armijo(alpha_try)

            x_try_old = x_cur + alpha_try_old*p_cur
            phi_try_old = f(x_try_old)

            if (phi_try > phi_armijo_try) | (phi_try > phi_try_old): # phi_try가 충분히 크다면 -> alpha_optm이 alpha_try_old와 alpha_try 사이 존재
                alpha_lo, alpha_hi = alpha_try_old, alpha_try
                alpha_optm = self.bracketing_alpha(f, f_torch, x_cur, p_cur, abs(c2*dphi0), phi_armijo, alpha_lo, alpha_hi) # bracketing 하고 interpolation iteration 돌려서 alpha_optm 뽑아내자
                return alpha_optm

            elif abs(dphi_try) <= -c2*dphi0: # phi_try가 충분히 작고 기울기까지 작다면
                alpha_optm = alpha_try # 그 점이 alph_optm이다
                return alpha_optm

            elif dphi_try >= 0: # phi_try가 충분히 작긴 한데 기울기가 양수라면 더 작은 phi 값을 가지는 alpha_optm이 alpha_try_old와 alpha_try 사이 존재
                alpha_lo, alpha_hi = alpha_try_old, alpha_try
                alpha_optm = self.bracketing_alpha(f, f_torch, x_cur, p_cur, abs(c2*dphi0), phi_armijo, alpha_lo, alpha_hi) # bracketing 하고 interpolation iteration 돌려서 alpha_optm 뽑아내자
                return alpha_optm

            else: # phi_try가 충분히 작긴 한데 기울기가 음수라면 더 작은 phi 값을 가지는 alpha_optm은 alpha_try보다 뒤의 구간에 존재 -> 구간 업데이트
                alpha_try_old = alpha_try # 다음 구간의 하한 = 현재 구간의 상한
                alpha_try = min(alpha_try * 2, 10.0) # 다음 구간의 상한 = 현재 구간 상한의 2배. 최대는 10으로 제한
        
        if not np.isfinite(alpha_try):
            alpha_try = 1e-3
        return max(min(alpha_try, 1.0), 1e-6)
    
    # Print log method(오버라이드 by 자식 메서드)
    def print_log(self):
        raise NotImplementedError # 반드시 자식 메서드에서 구현되어야 함을 개발자에게 표시

    # Solve - 부모 메서드인 solve()는 자식 클래스들이 공통으로 공유하는 optimization의 골격만 제공
    def solve(self):
        self.x_new = self.x
        self.f_new = self.f_val
        self.grad_f_new = self.grad_f_val

        for _ in range(self.max_iter):
            self.x_cur = self.x_new
            self.f_cur = self.f_new
            self.grad_f_cur = self.grad_f_new
            
            self.p_cur = self.search_direction()
            if self.grad_f_cur@self.p_cur > 0: # search direction이 증가 방향이면 경고 내보내고 steepest descent direction으로 search direction 변경
                print(f'Warning : grad(x_k)·p_k={self.grad_f_cur@self.p_cur} > 0 : Search direction p_k would likely make function increase !')
                print(f'Warning : p_k would be replaced with steepest descent direction grad(x_k) : {-self.grad_f_cur} !')
            
            alpha = self.wolfe_strong_interpol(self.f, self.f_torch, self.x_cur, self.f_cur, self.grad_f_cur, self.p_cur, c2=0.9)
            
            self.x_new = self.x_cur + alpha*self.p_cur
            self.f_new = self.f(self.x_new)
            self.grad_f_new = grad_torch(self.f_torch, self.x_new)
            self.r_grad_f_new = np.linalg.norm(self.grad_f_new)

            self.log_x.append(self.x_new)
            self.log_f_val.append(self.f_new)
            self.log_grad_f.append(self.grad_f_new)

            self.k += 1
            self.print_log()
            
            if self.r_grad_f_new <= self.tol:
                break
        
        optimum = self.x_new
        return optimum
# ----------------------------------------- Child Optimizer Class -----------------------------------------
class StpDec_torch(UnconOpt_torch):
    def search_direction(self):
        p = -grad_torch(self.f_torch, self.x_cur)
        return p

    def print_log(self):
        r_step = np.linalg.norm(self.x_new-self.x_cur)
        x_str = ", ".join([f"{xi:.8f}" for xi in self.x_new])
        print("\n log - Steepest Descent - torch")
        print(f"‖∆x‖ = {r_step:.2e}, "
            f"x{self.k+1:02d} = [{x_str}] | "
            f"f = {self.f_new:.4e}, "
            f"‖∇f‖ = {self.r_grad_f_new:.2e}, ")

### Jax AD 쓰는 버전

In [11]:
# 비제약 최적화에 대한 클래스화부터 한 후 제약 최적화에 대한 클래스화를 한다.
# ----------------------------------------- Parent Optimizer Class -----------------------------------------
class UnconOpt_jax: # 비제약 Optimizer(SDM, CGM, ...)에 대한 부모 클래스 정의
    '''
    자식 클래스들은 부모 클래스의 모든 함수 및 속성들을 상속받는다.  
    따라서 부모 클래스는 최대한 모든 자식 클래스에서 공통적으로 가지는 함수 및 속성들을 가지고 있어야 한다.  
    (바꿔 말하면, 기존에 클래스 없이 함수로만 짜여진 모듈을 클래스화하고 싶으면 그 함수들의 공통점을 최대한 솎아내서 부모 클래스가 가지도록 클래스를 설계해야 함)  
    자식 클래스는 부모 클래스에는 없는 자기 자신만의 함수 및 속성들을 가지고 있어야 한다(또는 부모 클래스의 함수를 이름만 같은 완전 새로운 자기만의 함수로 재정의할 수 있다).  
    '''
    def __init__(self, f, x0, tol=1e-6, max_iter=1000):
        '''
        Initialization method : 해당 클래스 인스턴스 생성(초기화) 시 부여되는 속성을 정의.  
        반드시 self 변수(해당 클래스의 인스턴스 생성 시 인스턴스 자기 자신을 의미)를 첫번째 arg로 받아야 함
        '''
        
        ### Check if x is 1D jax.Array
        if (not isinstance(x0, np.ndarray)) or (len(x0.shape) != 1):
            raise ValueError('Please input x0 as 1D np.ndarray !')
        else:
            self.x = x0.astype(np.float64)
        
        ### Check if f is callable
        if not callable(f):
            raise ValueError('Please input f as callable !')
        else:
            self.f = f
            self.grad_f = jax.jit(jax.grad(f))
        
        self.f_val = self.f(self.x)
        if not np.isfinite(self.f_val).all():
            raise ValueError('f(x0) is not finite. Check it again !')
        else:
            self.tol = float(tol)

        self.grad_f_val = self.grad_f(self.x)
        ### Check ||∇f(x0)||
        if np.linalg.norm(self.grad_f_val) < self.tol: # Check optimality
            print(f'Since ||grad(x0)|| = {np.linalg.norm(self.grad_f_val)} < {self.tol}, x0 : {self.x} is optimum point !')
            return [self.x], [self.f_val], [self.grad_f_val]
        else:
            print(f'Since ||grad(x0)|| = {np.linalg.norm(self.grad_f_val)} > {self.tol}, x0 : {self.x} is not an optimum point. Optimization available !')
            self.max_iter = max_iter
            self.k = 0
            pass

        # Optimization Log
        self.log_x = [self.x.copy()]
        self.log_f_val = [self.f_val.copy()]
        self.log_grad_f = [self.grad_f_val.copy()]

    # Search direction method(오버라이드 by 자식 메서드)
    def search_direction(self):
        raise NotImplementedError # 반드시 자식 메서드에서 구현되어야 함을 개발자에게 표시
    
    # interpol_alpha ⊂ bracketing_alpha ⊂ wolfe_strong_interpol
    def interpol_alpha(self, f, grad_f, x_cur, p_cur, a, b):
        # bracketing_alpha에서 계속해서 업데이트되는 구간(alpha_lo, alpha_hi)에 대해, 양단 점을 기반으로 2(3)차 함수로 근사하여 alpha_new 찾는 함수
        # 이 alpha_new는 bracketing_alpha에서 구간을 새로 업데이트할지 말지, 업데이트한다면 어떤 방향으로 업데이트할 지 판단할 때 사용
        phi_a = f(x_cur + a*p_cur)
        phi_b = f(x_cur + b*p_cur)
        dphi_a = grad_f(x_cur + a*p_cur)@p_cur
        
        dem = 2*(phi_b - phi_a - dphi_a*(b - a)) # 2차 함수 근사식 기반 minimum point 계산식의 분모
        
        if (abs(dem) < 1e-12) | (not np.isfinite(dem)): # 분모 수치적으로 불안정하면
            alpha_min = .5*(a + b) # 그냥 구간의 중심점을 minimum point로 상정하고 return
            return alpha_min
        else: # 분모 수치적으로 안정하면
            num = dphi_a*(b - a) # 분자 계산해주고
            alpha_min = a - num/dem # 2차근사식 minimum point 계산식 기반 minimum point 구해준다.
            alpha_min = np.clip(alpha_min, a + 0.1*(b - a), b - 0.1*(b - a)) # 만약 minimum point가 너무 작은 값이면(유의미한 point 이동 못 이뤄냄) 구간의 10% 지점으로, 너무 큰 값(너무 많이 point 이동해도 문제)이면 구간의 90% 지점으로 치환한다.
            return alpha_min   
    
    # bracketing_alpha ⊂ wolfe_strong_interpol
    def bracketing_alpha(self, f, grad_f, x_cur, p_cur, c2_dphi0, phi_armijo, alpha_lo, alpha_hi): 
        # wolfe_strong_interpol에서 확정된 '큰' 골짜기 구간을 기반으로 alpha_optm 찾는 함수
        # 여기서 추가적으로 구간을 더 작게 업데이트할 수도 있음 -> '작은' 골짜기 구간
        phi_lo = f(x_cur+alpha_lo*p_cur)

        for _ in range(50):
            alpha_new = self.interpol_alpha(f, grad_f, x_cur, p_cur, alpha_lo, alpha_hi) # 다항식 보간함수로 골짜기 구간에서의 최소추정점 alpha_new 구함
            phi_new = f(x_cur + alpha_new*p_cur) # alpha_new에서의 함수값
            dphi_new = grad_f(x_cur + alpha_new*p_cur)@p_cur # alpha_new에서의 기울기

            if (phi_new > phi_armijo(alpha_new)) | (phi_new >= phi_lo): # alpha_new에서의 함수값이 오히려 증가했다
                alpha_hi = alpha_new # alpha_optm은 alpha_lo와 alpha_new 사이 존재 -> '작은' 골짜기 구간 [alpha_lo, alpha_new]로 업뎃
                # alpha_lo unchanged
            else:
                if abs(dphi_new) <= c2_dphi0: # alpha_new에서의 함수값이 감소했고 기울기까지 작다
                    alpha_optm = alpha_new # alpha_new가 alpha_optm
                    return alpha_optm
                elif dphi_new > 0: # alpha_new에서의 함수값이 감소했는데 기울기는 여전히 양수다
                    alpha_hi = alpha_new # alpha_optm은 alpha_lo와 alpha_new 사이 존재 -> '작은' 골짜기 구간 [alpha_lo, alpha_new]로 업뎃
                    # alpha_lo unchanged
                else: # alpha_new에서의 함수값이 감소했는데 기울기가 음수다
                    alpha_lo = alpha_new # alpha_optm은 alpha_new와 alpha_hi 사이 존재 -> '작은' 골짜기 구간 [alpha_new, alpha_hi]로 업뎃
                    # alpha_hi unchanged
            
            phi_lo = f(x_cur + alpha_lo*p_cur) # 업뎃된 alpha_lo에서의 함수값 계산
            
            if abs(alpha_hi - alpha_lo) < 1e-8: # 만약 구간이 충분히 줄어들었으면 그냥 탈출해서 구간의 절반지점을 alpha_optm으로 return
                break
        
        alpha_optm = 0.5*(alpha_lo + alpha_hi)
        return alpha_optm

    # wolfe_strong_interpol - 확실한 '큰' 골짜기 구간을 찾아 거기서 alpha_optm을 찾는 함수
    def wolfe_strong_interpol(self, f, grad_f, x_cur, f_cur, grad_f_cur, p_cur, c2=0.9):
        c1 = 1e-4 # Armijo 조건, Curvature 조건용 factors
        alpha_try_old, alpha_try = 0, 1 # Initial bracket of alpha

        phi0 = f_cur # Armijo 함수 생성용
        dphi0 = grad_f_cur@p_cur # Armijo 함수 생성용 및 Curvature 조건 비교용

        phi_armijo = lambda alpha : phi0 + c1*alpha*dphi0 # Armijo 람다 함수 정의

        for _ in range(50):
            x_try = x_cur + alpha_try*p_cur
            phi_try = f(x_try)
            dphi_try = grad_f(x_try)@p_cur

            phi_armijo_try = phi_armijo(alpha_try)

            x_try_old = x_cur + alpha_try_old*p_cur
            phi_try_old = f(x_try_old)

            if (phi_try > phi_armijo_try) | (phi_try > phi_try_old): # phi_try가 충분히 크다면 -> alpha_optm이 alpha_try_old와 alpha_try 사이 존재
                alpha_lo, alpha_hi = alpha_try_old, alpha_try
                alpha_optm = self.bracketing_alpha(f, grad_f, x_cur, p_cur, abs(c2*dphi0), phi_armijo, alpha_lo, alpha_hi) # bracketing 하고 interpolation iteration 돌려서 alpha_optm 뽑아내자
                return alpha_optm

            elif abs(dphi_try) <= -c2*dphi0: # phi_try가 충분히 작고 기울기까지 작다면
                alpha_optm = alpha_try # 그 점이 alph_optm이다
                return alpha_optm

            elif dphi_try >= 0: # phi_try가 충분히 작긴 한데 기울기가 양수라면 더 작은 phi 값을 가지는 alpha_optm이 alpha_try_old와 alpha_try 사이 존재
                alpha_lo, alpha_hi = alpha_try_old, alpha_try
                alpha_optm = self.bracketing_alpha(f, grad_f, x_cur, p_cur, abs(c2*dphi0), phi_armijo, alpha_lo, alpha_hi) # bracketing 하고 interpolation iteration 돌려서 alpha_optm 뽑아내자
                return alpha_optm

            else: # phi_try가 충분히 작긴 한데 기울기가 음수라면 더 작은 phi 값을 가지는 alpha_optm은 alpha_try보다 뒤의 구간에 존재 -> 구간 업데이트
                alpha_try_old = alpha_try # 다음 구간의 하한 = 현재 구간의 상한
                alpha_try = min(alpha_try * 2, 10.0) # 다음 구간의 상한 = 현재 구간 상한의 2배. 최대는 10으로 제한
        
        if not np.isfinite(alpha_try):
            alpha_try = 1e-3
        return max(min(alpha_try, 1.0), 1e-6)
    
    # Print log method(오버라이드 by 자식 메서드)
    def print_log(self):
        raise NotImplementedError # 반드시 자식 메서드에서 구현되어야 함을 개발자에게 표시

    # Solve - 부모 메서드인 solve()는 자식 클래스들이 공통으로 공유하는 optimization의 골격만 제공
    def solve(self):
        self.x_new = self.x
        self.f_new = self.f_val
        self.grad_f_new = self.grad_f_val

        for _ in range(self.max_iter):
            self.x_cur = self.x_new
            self.f_cur = self.f_new
            self.grad_f_cur = self.grad_f_new
            
            self.p_cur = self.search_direction()
            if self.grad_f_cur@self.p_cur > 0: # search direction이 증가 방향이면 경고 내보내고 steepest descent direction으로 search direction 변경
                print(f'Warning : grad(x_k)·p_k={self.grad_f_cur@self.p_cur} > 0 : Search direction p_k would likely make function increase !')
                print(f'Warning : p_k would be replaced with steepest descent direction grad(x_k) : {-self.grad_f_cur} !')
            
            alpha = self.wolfe_strong_interpol(self.f, self.grad_f, self.x_cur, self.f_cur, self.grad_f_cur, self.p_cur, c2=0.9)
            
            self.x_new = self.x_cur + alpha*self.p_cur
            self.f_new = self.f(self.x_new)
            self.grad_f_new = self.grad_f(self.x_new)
            self.r_grad_f_new = np.linalg.norm(self.grad_f_new)

            self.log_x.append(self.x_new)
            self.log_f_val.append(self.f_new)
            self.log_grad_f.append(self.grad_f_new)

            self.k += 1
            self.print_log()
            
            if self.r_grad_f_new <= self.tol:
                break
        
        optimum = self.x_new
        return optimum
# ----------------------------------------- Child Optimizer Class -----------------------------------------
class StpDec_jax(UnconOpt_jax):
    def search_direction(self):
        p = -self.grad_f(self.x_cur)
        return p

    def print_log(self):
        r_step = np.linalg.norm(self.x_new-self.x_cur)
        x_str = ", ".join([f"{xi:.8f}" for xi in self.x_new])
        print("\n log - Steepest Descent - jax")
        print(f"‖∆x‖ = {r_step:.2e}, "
            f"x{self.k+1:02d} = [{x_str}] | "
            f"f = {self.f_new:.4e}, "
            f"‖∇f‖ = {self.r_grad_f_new:.2e}, ")

### 내가 만든 FD 쓰는 버전

In [12]:
def grad_centraldiff(f, x):
    x = np.atleast_1d(x)
    rel_step = 1e-6
    dfdx = np.zeros(len(x))
    for i in range(len(x)):
        h = rel_step*np.max([np.abs(x[i]), 1])
        e_unit = np.zeros(len(x)); e_unit[i] = 1
        dx = h*e_unit
        num = f(x+dx) - f(x-dx)
        den = 2*h
        dfdx[i] = num/den
    if not np.isfinite(dfdx).all(): # Check finitude
        raise ValueError('At least one component of gradient is not finite !')
    return dfdx

# 비제약 최적화에 대한 클래스화부터 한 후 제약 최적화에 대한 클래스화를 한다.
# ----------------------------------------- Parent Optimizer Class -----------------------------------------
class UnconOpt_FD: # 비제약 Optimizer(SDM, CGM, ...)에 대한 부모 클래스 정의
    '''
    자식 클래스들은 부모 클래스의 모든 함수 및 속성들을 상속받는다.  
    따라서 부모 클래스는 최대한 모든 자식 클래스에서 공통적으로 가지는 함수 및 속성들을 가지고 있어야 한다.  
    (바꿔 말하면, 기존에 클래스 없이 함수로만 짜여진 모듈을 클래스화하고 싶으면 그 함수들의 공통점을 최대한 솎아내서 부모 클래스가 가지도록 클래스를 설계해야 함)  
    자식 클래스는 부모 클래스에는 없는 자기 자신만의 함수 및 속성들을 가지고 있어야 한다(또는 부모 클래스의 함수를 이름만 같은 완전 새로운 자기만의 함수로 재정의할 수 있다).  
    '''
    def __init__(self, f, x0, tol=1e-6, max_iter=1000):
        '''
        Initialization method : 해당 클래스 인스턴스 생성(초기화) 시 부여되는 속성을 정의.  
        반드시 self 변수(해당 클래스의 인스턴스 생성 시 인스턴스 자기 자신을 의미)를 첫번째 arg로 받아야 함
        '''
        if not callable(f):
            raise ValueError('Please input f as function type !')
        else:
            self.f = f

        if (not isinstance(x0, np.ndarray)) or (len(x0.shape) != 1):
            raise ValueError('Please input x0 as 1D ndarray type !')
        else:
            self.x = x0.astype(np.float64)
        
        self.f_val = self.f(self.x)
        if not np.isfinite(self.f_val).all():
            raise ValueError('f(x0) is not finite. Check it again !')
        else:
            self.tol = float(tol)

        self.grad_f_val = grad_centraldiff(self.f, self.x)
        ### Check ||∇f(x0)||
        if np.linalg.norm(self.grad_f_val) < self.tol: # Check optimality
            print(f'Since ||grad(x0)|| = {np.linalg.norm(self.grad_f_val)} < {self.tol}, x0 : {self.x} is optimum point !')
            return [self.x], [self.f_val], [self.grad_f_val]
        else:
            print(f'Since ||grad(x0)|| = {np.linalg.norm(self.grad_f_val)} > {self.tol}, x0 : {self.x} is not an optimum point. Optimization available !')
            self.max_iter = max_iter
            self.k = 0
            pass

        # Optimization Log
        self.log_x = [self.x.copy()]
        self.log_f_val = [self.f_val.copy()]
        self.log_grad_f = [self.grad_f_val.copy()]

    # Search direction method(오버라이드 by 자식 메서드)
    def search_direction(self):
        raise NotImplementedError # 반드시 자식 메서드에서 구현되어야 함을 개발자에게 표시
    
    # interpol_alpha ⊂ bracketing_alpha ⊂ wolfe_strong_interpol
    def interpol_alpha(self, f, x_cur, p_cur, a, b):
        # bracketing_alpha에서 계속해서 업데이트되는 구간(alpha_lo, alpha_hi)에 대해, 양단 점을 기반으로 2(3)차 함수로 근사하여 alpha_new 찾는 함수
        # 이 alpha_new는 bracketing_alpha에서 구간을 새로 업데이트할지 말지, 업데이트한다면 어떤 방향으로 업데이트할 지 판단할 때 사용
        phi_a = f(x_cur + a*p_cur)
        phi_b = f(x_cur + b*p_cur)
        dphi_a = grad_centraldiff(f, x_cur + a*p_cur)@p_cur
        
        dem = 2*(phi_b - phi_a - dphi_a*(b - a)) # 2차 함수 근사식 기반 minimum point 계산식의 분모
        
        if (abs(dem) < 1e-12) | (not np.isfinite(dem)): # 분모 수치적으로 불안정하면
            alpha_min = .5*(a + b) # 그냥 구간의 중심점을 minimum point로 상정하고 return
            return alpha_min
        else: # 분모 수치적으로 안정하면
            num = dphi_a*(b - a) # 분자 계산해주고
            alpha_min = a - num/dem # 2차근사식 minimum point 계산식 기반 minimum point 구해준다.
            alpha_min = np.clip(alpha_min, a + 0.1*(b - a), b - 0.1*(b - a)) # 만약 minimum point가 너무 작은 값이면(유의미한 point 이동 못 이뤄냄) 구간의 10% 지점으로, 너무 큰 값(너무 많이 point 이동해도 문제)이면 구간의 90% 지점으로 치환한다.
            return alpha_min   
    
    # bracketing_alpha ⊂ wolfe_strong_interpol
    def bracketing_alpha(self, f, x_cur, p_cur, c2_dphi0, phi_armijo, alpha_lo, alpha_hi): 
        # wolfe_strong_interpol에서 확정된 '큰' 골짜기 구간을 기반으로 alpha_optm 찾는 함수
        # 여기서 추가적으로 구간을 더 작게 업데이트할 수도 있음 -> '작은' 골짜기 구간
        phi_lo = f(x_cur+alpha_lo*p_cur)

        for _ in range(50):
            alpha_new = self.interpol_alpha(f, x_cur, p_cur, alpha_lo, alpha_hi) # 다항식 보간함수로 골짜기 구간에서의 최소추정점 alpha_new 구함
            phi_new = f(x_cur + alpha_new*p_cur) # alpha_new에서의 함수값
            dphi_new = grad_centraldiff(f, x_cur + alpha_new*p_cur)@p_cur # alpha_new에서의 기울기

            if (phi_new > phi_armijo(alpha_new)) | (phi_new >= phi_lo): # alpha_new에서의 함수값이 오히려 증가했다
                alpha_hi = alpha_new # alpha_optm은 alpha_lo와 alpha_new 사이 존재 -> '작은' 골짜기 구간 [alpha_lo, alpha_new]로 업뎃
                # alpha_lo unchanged
            else:
                if abs(dphi_new) <= c2_dphi0: # alpha_new에서의 함수값이 감소했고 기울기까지 작다
                    alpha_optm = alpha_new # alpha_new가 alpha_optm
                    return alpha_optm
                elif dphi_new > 0: # alpha_new에서의 함수값이 감소했는데 기울기는 여전히 양수다
                    alpha_hi = alpha_new # alpha_optm은 alpha_lo와 alpha_new 사이 존재 -> '작은' 골짜기 구간 [alpha_lo, alpha_new]로 업뎃
                    # alpha_lo unchanged
                else: # alpha_new에서의 함수값이 감소했는데 기울기가 음수다
                    alpha_lo = alpha_new # alpha_optm은 alpha_new와 alpha_hi 사이 존재 -> '작은' 골짜기 구간 [alpha_new, alpha_hi]로 업뎃
                    # alpha_hi unchanged
            
            phi_lo = f(x_cur + alpha_lo*p_cur) # 업뎃된 alpha_lo에서의 함수값 계산
            
            if abs(alpha_hi - alpha_lo) < 1e-8: # 만약 구간이 충분히 줄어들었으면 그냥 탈출해서 구간의 절반지점을 alpha_optm으로 return
                break
        
        alpha_optm = 0.5*(alpha_lo + alpha_hi)
        return alpha_optm

    # wolfe_strong_interpol - 확실한 '큰' 골짜기 구간을 찾아 거기서 alpha_optm을 찾는 함수
    def wolfe_strong_interpol(self, f, x_cur, f_cur, grad_f_cur, p_cur, c2=0.9):
        c1 = 1e-4 # Armijo 조건, Curvature 조건용 factors
        alpha_try_old, alpha_try = 0, 1 # Initial bracket of alpha

        phi0 = f_cur # Armijo 함수 생성용
        dphi0 = grad_f_cur@p_cur # Armijo 함수 생성용 및 Curvature 조건 비교용

        phi_armijo = lambda alpha : phi0 + c1*alpha*dphi0 # Armijo 람다 함수 정의

        for _ in range(50):
            x_try = x_cur + alpha_try*p_cur
            phi_try = f(x_try)
            dphi_try = grad_centraldiff(f, x_try)@p_cur

            phi_armijo_try = phi_armijo(alpha_try)

            x_try_old = x_cur + alpha_try_old*p_cur
            phi_try_old = f(x_try_old)

            if (phi_try > phi_armijo_try) | (phi_try > phi_try_old): # phi_try가 충분히 크다면 -> alpha_optm이 alpha_try_old와 alpha_try 사이 존재
                alpha_lo, alpha_hi = alpha_try_old, alpha_try
                alpha_optm = self.bracketing_alpha(f, x_cur, p_cur, abs(c2*dphi0), phi_armijo, alpha_lo, alpha_hi) # bracketing 하고 interpolation iteration 돌려서 alpha_optm 뽑아내자
                return alpha_optm

            elif abs(dphi_try) <= -c2*dphi0: # phi_try가 충분히 작고 기울기까지 작다면
                alpha_optm = alpha_try # 그 점이 alph_optm이다
                return alpha_optm

            elif dphi_try >= 0: # phi_try가 충분히 작긴 한데 기울기가 양수라면 더 작은 phi 값을 가지는 alpha_optm이 alpha_try_old와 alpha_try 사이 존재
                alpha_lo, alpha_hi = alpha_try_old, alpha_try
                alpha_optm = self.bracketing_alpha(f, x_cur, p_cur, abs(c2*dphi0), phi_armijo, alpha_lo, alpha_hi) # bracketing 하고 interpolation iteration 돌려서 alpha_optm 뽑아내자
                return alpha_optm

            else: # phi_try가 충분히 작긴 한데 기울기가 음수라면 더 작은 phi 값을 가지는 alpha_optm은 alpha_try보다 뒤의 구간에 존재 -> 구간 업데이트
                alpha_try_old = alpha_try # 다음 구간의 하한 = 현재 구간의 상한
                alpha_try = min(alpha_try * 2, 10.0) # 다음 구간의 상한 = 현재 구간 상한의 2배. 최대는 10으로 제한
        
        if not np.isfinite(alpha_try):
            alpha_try = 1e-3
        return max(min(alpha_try, 1.0), 1e-6)
    
    # Print log method(오버라이드 by 자식 메서드)
    def print_log(self):
        raise NotImplementedError # 반드시 자식 메서드에서 구현되어야 함을 개발자에게 표시

    # Solve - 부모 메서드인 solve()는 자식 클래스들이 공통으로 공유하는 optimization의 골격만 제공
    def solve(self):
        self.x_new = self.x
        self.f_new = self.f_val
        self.grad_f_new = self.grad_f_val

        for _ in range(self.max_iter):
            self.x_cur = self.x_new
            self.f_cur = self.f_new
            self.grad_f_cur = self.grad_f_new
            
            self.p_cur = self.search_direction()
            if self.grad_f_cur@self.p_cur > 0: # search direction이 증가 방향이면 경고 내보내고 steepest descent direction으로 search direction 변경
                print(f'Warning : grad(x_k)·p_k={self.grad_f_cur@self.p_cur} > 0 : Search direction p_k would likely make function increase !')
                print(f'Warning : p_k would be replaced with steepest descent direction grad(x_k) : {-self.grad_f_cur} !')
            
            alpha = self.wolfe_strong_interpol(self.f, self.x_cur, self.f_cur, self.grad_f_cur, self.p_cur, c2=0.9)
            
            self.x_new = self.x_cur + alpha*self.p_cur
            self.f_new = self.f(self.x_new)
            self.grad_f_new = grad_centraldiff(self.f, self.x_new)
            self.r_grad_f_new = np.linalg.norm(self.grad_f_new)

            self.log_x.append(self.x_new)
            self.log_f_val.append(self.f_new)
            self.log_grad_f.append(self.grad_f_new)

            self.k += 1
            self.print_log()
            
            if self.r_grad_f_new <= self.tol:
                break
        
        optimum = self.x_new
        return optimum
# ----------------------------------------- Child Optimizer Class -----------------------------------------
class StpDec_FD(UnconOpt_FD):
    def search_direction(self):
        p = -grad_centraldiff(self.f, self.x_cur)
        return p

    def print_log(self):
        r_step = np.linalg.norm(self.x_new-self.x_cur)
        x_str = ", ".join([f"{xi:.8f}" for xi in self.x_new])
        print("\n log - Steepest Descent - FD")
        print(f"‖∆x‖ = {r_step:.2e}, "
            f"x{self.k+1:02d} = [{x_str}] | "
            f"f = {self.f_new:.4e}, "
            f"‖∇f‖ = {self.r_grad_f_new:.2e}, ")

### 클래스 테스트

In [ ]:
# Test function 1 : Sphere function
func_sphere = lambda x : x[0]**2 + x[1]**2
Dfunc_sphere = lambda x : np.array([2*x[0], 
                                    2*x[1]]).reshape(-1, 1) # reshape(-1, 1) --> ?행 1열짜리 행렬로 바꿔줘. ?행이 될지는 니가 알아서 계산해서 처리해줘.라는 명령어

# Test function 2 : Beale Function
func_beale = lambda x : (1.5 - x[0] + x[0]*x[1])**2 + (2.25 - x[0] + x[0]*x[1]**2)**2 + (2.625 - x[0] + x[0]*x[1]**3)**2
Dfunc_beale = lambda x : np.array([2*(1.5 - x[0] + x[0]*x[1])*(-1 + x[1]) + 2*(2.25 - x[0] + x[0]*x[1]**2)*(-1 + x[1]**2) + 2*(2.625 - x[0] + x[0]*x[1]**3)*(-1 + x[1]**3),
                                    2*x[0]*(1.5 - x[0] + x[0]*x[1]) + 4*x[0]*x[1]*(2.25 - x[0] + x[0]*x[1]**2) + 6*x[0]*x[1]**2*(2.625 - x[0] + x[0]*x[1]**3)]).reshape(-1, 1)

# Test function 3 : Booth Function
func_booth = lambda x : (x[0] + 2*x[1] - 7)**2 + (2*x[0] + x[1] - 5)**2
Dfunc_booth = lambda x : np.array([2*(x[0] + 2*x[1] - 7) + 4*(2*x[0] + x[1] - 5),
                                   4*(x[0] + 2*x[1] - 7) + 2*(2*x[0] + x[1] - 5)]).reshape(-1, 1)

# Test function 4 : Matyas Function
func_matyas = lambda x : 0.26*(x[0]**2 + x[1]**2) - 0.48*x[0]*x[1]
Dfunc_matyas = lambda x : np.array([0.52*x[0] - 0.48*x[1], 
                                    0.52*x[1] - 0.48*x[0]]).reshape(-1, 1)

# Target function
func_rosenbrock = lambda x : (1.0 - x[0])**2 + 100*(x[1] - x[0]**2)**2
Dfunc_rosenbrock = lambda x : np.array([-2*(1 - x[0]) - 400*x[0]*(x[1] - x[0]**2),
                                        200*(x[1] - x[0]**2)]).reshape(-1, 1)

In [ ]:
# optimziation setting
obj = func_rosenbrock
tol = 1e-2
x0 = np.array([0.0, 3.0])

# solve - FD > torch(AD) > jax(AD) 순으로 빠름 ... jax 맞춤형 문법으로 jax 버전 StpDec 코딩 시 가장 빠르게 문제를 풀 수 있으나 너무 변태스럽게 바꿔야 되서 어려움.
optimizer_FD = StpDec_FD(f=obj, x0=x0, tol=tol, max_iter=20000)
optimizer_FD.solve()

optimizer_torch = StpDec_torch(f=obj, f_torch=obj, x0=x0, tol=tol, max_iter=20000)
optimizer_torch.solve()

optimizer_jax = StpDec_jax(f=obj, x0=x0, tol=tol, max_iter=20000)
optimizer_jax.solve()

Since ||grad(x0)|| = 600.0033333240741 > 0.01, x0 : [0. 3.] is not an optimum point. Optimization available !

 log - Steepest Descent - torch
‖∆x‖ = 5.55e+00, x02 = [0.01850696, -2.55208661] | f = 6.5245e+02, ‖∇f‖ = 5.11e+02, 

 log - Steepest Descent - torch
‖∆x‖ = 4.74e+00, x03 = [-0.13875677, 2.18925952] | f = 4.7219e+02, ‖∇f‖ = 4.50e+02, 

 log - Steepest Descent - torch
‖∆x‖ = 2.66e+00, x04 = [-0.83650160, -0.37347426] | f = 1.1855e+02, ‖∇f‖ = 4.22e+02, 

 log - Steepest Descent - torch
‖∆x‖ = 2.49e+00, x05 = [1.30561594, 0.89396433] | f = 6.5812e+01, ‖∇f‖ = 4.54e+02, 

 log - Steepest Descent - torch
‖∆x‖ = 2.98e+00, x06 = [-1.47611678, 1.95772372] | f = 1.1024e+01, ‖∇f‖ = 1.43e+02, 

 log - Steepest Descent - torch
‖∆x‖ = 1.40e-01, x07 = [-1.34262391, 2.00128940] | f = 9.4341e+00, ‖∇f‖ = 1.09e+02, 

 log - Steepest Descent - torch
‖∆x‖ = 1.08e-01, x08 = [-1.44307043, 1.96216436] | f = 7.4155e+00, ‖∇f‖ = 7.81e+01, 

 log - Steepest Descent - torch
‖∆x‖ = 7.69e-02, x09 = [-1.3698

array([0.99698812, 0.99395167])

In [ ]:
# Torch AD 버전 optimization 위한 test function(np 연산 버전 / torch 연산 버전)
# mishra function의 x* : (-3.1302468, -1.5821422)
f_mishra_np = lambda x : np.sin(x[1])*np.exp((1-np.cos(x[0]))**2) + np.cos(x[0])*np.exp((1-np.sin(x[1]))**2) + (x[0] - x[1])**2
f_mishra_torch = lambda x : torch.sin(x[1])*torch.exp((1-torch.cos(x[0]))**2) + torch.cos(x[0])*torch.exp((1-torch.sin(x[1]))**2) + (x[0] - x[1])**2

# Numpy 연산 기반의 함수 f_np와 Torch 연산 기반의 함수 f_torch(torch의 AD evaluation 위함)를 따로 넣어주어 StpDec 성공적 수행 확인
x0 = np.array([-4.0, -2.0])
tol = 1e-2
optimizer_torch = StpDec_torch(f=f_mishra_np, f_torch=f_mishra_torch, x0=x0, tol=tol, max_iter=20000)
optimizer_torch.solve()

Since ||grad(x0)|| = 80.0593234385669 > 0.01, x0 : [-4. -2.] is not an optimum point. Optimization available !

 log - Steepest Descent - torch
‖∆x‖ = 1.26e+00, x02 = [-2.93263720, -1.33814163] | f = -9.4169e+01, ‖∇f‖ = 7.49e+01, 

 log - Steepest Descent - torch
‖∆x‖ = 5.35e-01, x03 = [-3.26826889, -1.75536279] | f = -1.0031e+02, ‖∇f‖ = 5.62e+01, 

 log - Steepest Descent - torch
‖∆x‖ = 3.91e-01, x04 = [-3.02073870, -1.45306379] | f = -1.0293e+02, ‖∇f‖ = 4.44e+01, 

 log - Steepest Descent - torch
‖∆x‖ = 3.10e-01, x05 = [-3.21979954, -1.69029230] | f = -1.0411e+02, ‖∇f‖ = 3.72e+01, 

 log - Steepest Descent - torch
‖∆x‖ = 2.49e-01, x06 = [-3.05995504, -1.49974658] | f = -1.0518e+02, ‖∇f‖ = 2.91e+01, 

 log - Steepest Descent - torch
‖∆x‖ = 2.01e-01, x07 = [-3.18978740, -1.65331908] | f = -1.0560e+02, ‖∇f‖ = 2.50e+01, 

 log - Steepest Descent - torch
‖∆x‖ = 1.61e-01, x08 = [-3.08632887, -1.53047789] | f = -1.0614e+02, ‖∇f‖ = 1.84e+01, 

 log - Steepest Descent - torch
‖∆x‖ = 1.17e-01,

array([-3.1302311 , -1.58211171])